In [1]:
from Fundamentals import Dirichlet, PPEProbabilities
import numpy as np
import pandas as pd
import scipy.stats as scs
import pymc as pm

### Some simple checks regarding the Dirichlet class

In [2]:
dir_1 = Dirichlet(1)

incorrect_probs = np.array([0.5,0.1,0.2,0.3])
expert_probs = np.array([0.2,0.15,0.25,0.4])

## Gives an error, as it should

##print(dir_1.alpha_mle(incorrect_probs, expert_probs))

        
probs = np.array([0.3,0.05,0.2,0.45])
expert_probs = np.array([0.2,0.15,0.25,0.4])


print(dir_1.pdf(probs, expert_probs))
print(dir_1.llik(probs, expert_probs))
print(dir_1.alpha_mle(probs, expert_probs))


dir_2 = Dirichlet(None) 
dir_3 = Dirichlet(dir_1.alpha_mle(probs, expert_probs))

##checking that if alpha is None, then it is computed based on dirichlet mle

print(dir_2.llik(probs, expert_probs) == dir_3.llik(probs, expert_probs)) 

print("-----------------")

sample_incorrect_probs = [[0.3,0.05,0.2,0.45], [0.35,0.05,0.2,0.45]]
sample_expert_probs = [[0.2,0.15,0.25,0.4], [0.2,0.15,0.25,0.4]]

## Gives an error

## print(dir_1.sum_llik(sample_incorrect_probs, sample_expert_probs))



sample_probs = [np.array([0.3,0.05,0.2,0.45]), np.array([0.15,0.25,0.5,0.1])]
sample_expert_probs = [np.array([0.2,0.15,0.25,0.4]), np.array([0.1,0.2,0.5,0.2])]

print(dir_1.sum_llik(sample_probs, sample_expert_probs))
print(dir_1.alpha_mle(sample_probs, sample_expert_probs))

print("----------------")


dir_4 = Dirichlet(dir_1.alpha_mle(sample_probs, sample_expert_probs))

##checking that if alpha is None, then it is computed based on dirichlet mle

print(dir_4.sum_llik(sample_probs, sample_expert_probs) == dir_2.sum_llik(sample_probs, sample_expert_probs)) 

0.17835806
-1.7239618
19.97801
True
-----------------
-2.7055316
24.51511
----------------
True


In [3]:
## Checking that having different probability dimensions is ok (necessary for having different number of partitions)

dir_1 = Dirichlet(None) 


sample_probs = [np.array([0.3,0.05,0.2,0.45]), np.array([0.3, 0.25, 0.45])] ## 4 and 3 probabilities (partitions)
sample_expert_probs = [np.array([0.2,0.15,0.25,0.4]), np.array([0.2, 0.7, 0.1])]



print(dir_1.sum_llik(sample_probs, sample_expert_probs))
print(dir_1.alpha_mle(sample_probs, sample_expert_probs))

0.8049607
4.0574403


# Running some tests to ensure that the two classes work as expected

## Discrete data

In [4]:
############### Getting the data from a folder ###############

probs_1 = np.array([[0, 1, 2], [0.5, 0.3, 0.2]]).T
probs_2 = np.array([[0, 1, 2], [0.4, 0.4, 0.2]]).T
probs_3 = np.array([[0, 1, 2], [0.3, 0.1, 0.6]]).T

# Folder path to store CSV files
folder_path = '/Users/panos/Desktop/Internship/PPE_Python/Random_test_files/test_folder'

probs_1 = pd.DataFrame(probs_1)
probs_2 = pd.DataFrame(probs_2)
probs_3 = pd.DataFrame(probs_3)

probs_1.to_csv(folder_path + "/probs_1.csv")
probs_2.to_csv(folder_path + "/probs_2.csv")
probs_3.to_csv(folder_path + "/probs_3.csv")

prob_class = PPEProbabilities(target_type="discrete", path=True)


partitions, expert_probs = prob_class.get_expert_data(folder_path)

print(partitions)
print(expert_probs)


[0. 1. 2.]
[array([0.3, 0.1, 0.6]), array([0.4, 0.4, 0.2]), array([0.5, 0.3, 0.2])]


In [5]:
############### Getting the data from a file ###############

probs_file = np.array([[0, 1, 2], [0.5, 0.3, 0.2], [0.4, 0.4, 0.2], [0.3, 0.1, 0.6]]).T

# Path to store the CSV file

probs_file = pd.DataFrame(probs_file)

probs_file.to_csv('/Users/panos/Desktop/Internship/PPE_Python/Random_test_files/probs_file.csv')

prob_class = PPEProbabilities(target_type="discrete", path=True)


partitions, expert_probs = prob_class.get_expert_data('/Users/panos/Desktop/Internship/PPE_Python/Random_test_files/probs_file.csv')

print(partitions)
print(expert_probs)

[0. 1. 2.]
[array([0.5, 0.3, 0.2]), array([0.4, 0.4, 0.2]), array([0.3, 0.1, 0.6])]


In [6]:
############### Feeding the data directly ###############
 

probs = np.array([[0, 1, 2], [0.5, 0.3, 0.2], [0.4, 0.4, 0.2], [0.3, 0.1, 0.6]]).T

prob_class = PPEProbabilities(target_type="discrete", path=False)


partitions, expert_probs = prob_class.get_expert_data(probs)

print(partitions)
print(expert_probs)

[0. 1. 2.]
[array([0.5, 0.3, 0.2]), array([0.4, 0.4, 0.2]), array([0.3, 0.1, 0.6])]


In [7]:
######## Test where we input samples from a simple prior predictive distribution and get model probabilities for the partitions ########

elicited_data_discrete = np.array([[0,1],[0.3,0.7],[0.6,0.4]]).T


cov_set_1 = np.random.binomial(n = 1, p = 0.7, size = 2000)
cov_set_2 = np.random.binomial(n = 1, p = 0.4, size = 2000)

samples = np.vstack((cov_set_1, cov_set_2)).T

prob_class = PPEProbabilities(target_type="discrete", path=False)


partitions, expert_probs = prob_class.get_expert_data(elicited_data_discrete)

model_probs = prob_class.ppd_probs(samples, partitions)

print(partitions)
print(expert_probs)
print(model_probs)
print("------------")

## Feeding these probabilities to dirichlet

dir = Dirichlet(None)

print(dir.alpha_mle(model_probs, expert_probs))  ## very high alpha, which makes sense since we used the "expert" probabilities to sample

print(dir.sum_llik(model_probs, expert_probs))

dir_2 = Dirichlet(10) ## trying fixed alpha

print(dir_2.sum_llik(model_probs, expert_probs))

[0. 1.]
[array([0.3, 0.7]), array([0.6, 0.4])]
[array([0.29, 0.71]), array([0.595, 0.405])]
------------
3429.006
6.7924805
1.8896246


## Continuous data

In [8]:
############### Getting the data from a folder ###############

probs_1 = np.array([[0, 100, 200], [100, 200, 300], [0.5, 0.3, 0.2]]).T  ## partitions (0,100), (100, 200), (200, 300)
probs_2 = np.array([[0, 150, 200], [150, 200, 300], [0.4, 0.4, 0.2]]).T  ## partitions (0,150), (150, 200), (200, 300)
probs_3 = np.array([[0, 100, 150, 200], [100, 150, 200, 300], [0.3, 0.1, 0.4, 0.2]]).T   ## Different number of partitions here! partitions (0,100), (100, 150), (150, 200), (200, 300)

# Folder path to store CSV files
folder_path = '/Users/panos/Desktop/Internship/PPE_Python/Random_test_files/test_folder_2'

probs_1 = pd.DataFrame(probs_1)
probs_2 = pd.DataFrame(probs_2)
probs_3 = pd.DataFrame(probs_3)

probs_1.to_csv(folder_path + "/probs_1.csv")
probs_2.to_csv(folder_path + "/probs_2.csv")
probs_3.to_csv(folder_path + "/probs_3.csv")

prob_class = PPEProbabilities(target_type="continuous", path=True)


partitions, expert_probs = prob_class.get_expert_data(folder_path)

for partition in partitions:
    print(partition)

print(expert_probs)


[[  0. 100.]
 [100. 150.]
 [150. 200.]
 [200. 300.]]
[[  0. 150.]
 [150. 200.]
 [200. 300.]]
[[  0. 100.]
 [100. 200.]
 [200. 300.]]
[array([0.3, 0.1, 0.4, 0.2]), array([0.4, 0.4, 0.2]), array([0.5, 0.3, 0.2])]


In [9]:
############### Feeding the data directly ###############

probs_1 = np.array([[0, 100, 200], [100, 200, 300], [0.5, 0.3, 0.2]]).T
probs_2 = np.array([[0, 150, 200], [150, 200, 300], [0.4, 0.4, 0.2]]).T
probs_3 = np.array([[0, 100, 150, 200], [100, 150, 200, 300], [0.3, 0.1, 0.4, 0.2]]).T   ## Different number of partitions here!

probs = [probs_1, probs_2, probs_3]

prob_class = PPEProbabilities(target_type="continuous", path=False)


partitions, expert_probs = prob_class.get_expert_data(probs)

for partition in partitions:
    print(partition)

print(expert_probs)

[[  0. 100.]
 [100. 200.]
 [200. 300.]]
[[  0. 150.]
 [150. 200.]
 [200. 300.]]
[[  0. 100.]
 [100. 150.]
 [150. 200.]
 [200. 300.]]
[array([0.5, 0.3, 0.2]), array([0.4, 0.4, 0.2]), array([0.3, 0.1, 0.4, 0.2])]


In [10]:
######## Test where we input samples from a simple prior predictive distribution and get model probabilities for the partitions ########

probs_1 = np.array([[0, 100, 200], [100, 200, 300], [0.5, 0.3, 0.2]]).T
probs_2 = np.array([[0, 150, 200], [150, 200, 300], [0.4, 0.4, 0.2]]).T
probs_3 = np.array([[0, 100, 150, 200], [100, 150, 200, 300], [0.3, 0.1, 0.4, 0.2]]).T   ## Different number of partitions here!

probs = [probs_1, probs_2, probs_3]





samples_1 = np.random.normal(loc = 66, scale = 45, size = 10000)
samples_2 = np.random.normal(loc = 150, scale = 35, size = 10000)
samples_3 = np.random.normal(loc = 200, scale = 35, size = 10000)

samples = np.vstack((samples_1, samples_2, samples_3)).T


prob_class = PPEProbabilities(target_type="continuous", path=False)


partitions, expert_probs = prob_class.get_expert_data(probs)

model_probs = prob_class.ppd_probs(samples, partitions)

print(partitions)
print(expert_probs)
print(model_probs)
print("------------")



dir = Dirichlet(None)

print(dir.alpha_mle(model_probs, expert_probs))  ## low alpha, which makes sense since we used a "random" distribution to sample, loosely following the expert distribution

print(dir.sum_llik(model_probs, expert_probs))

dir_2 = Dirichlet(10) ## trying fixed alpha

print(dir_2.sum_llik(model_probs, expert_probs))


[array([[  0., 100.],
       [100., 200.],
       [200., 300.]]), array([[  0., 150.],
       [150., 200.],
       [200., 300.]]), array([[  0., 100.],
       [100., 150.],
       [150., 200.],
       [200., 300.]])]
[array([0.5, 0.3, 0.2]), array([0.4, 0.4, 0.2]), array([0.3, 0.1, 0.4, 0.2])]
[array([0.7774, 0.2209, 0.0017]), array([0.5003, 0.4233, 0.0764]), array([0.0017, 0.0748, 0.4198, 0.5037])]
------------
4.459805
-6.878395
-7.330767


## Running some more complicated examples

### Example 1 (from paper): Univariate Gaussian

We assume $ Y \sim \mathcal{N}(\theta, \sigma)$, with $\theta \sim \frac{1}{2} \mathcal{N}(\mu_1, \sigma_1) + \frac{1}{2} \mathcal{N}(\mu_2, \sigma_2)$. Then, we have the hyperparameter vector $\pmb{\lambda} = [\mu_1, \mu_2, \sigma, \sigma_1, \sigma_2]$. Also, for $ A = (a,b] $, we know that 

$$\mathbb{P}_{A|\pmb{\lambda}} = \sum_{k=1}^{2} \Big( \frac{1}{2} \Phi \Big((b - \mu_k)/\sqrt{\sigma^2 + \sigma_k^2} \Big) - \frac{1}{2} \Phi \Big((a - \mu_k)/\sqrt{\sigma^2 + \sigma_k^2} \Big) \Big)$$

In [11]:
mu_1 = 1
mu_2 = -1
sigma = sigma_1 = sigma_2 = 1

def get_gaussian_probs(partition):
    
    p1 = 0.5*(scs.norm.cdf((partition[1] - mu_1)/np.sqrt(sigma**2 + sigma_1**2)) - scs.norm.cdf((partition[0] - mu_1)/np.sqrt(sigma**2 + sigma_1**2)))
    p2 = 0.5*(scs.norm.cdf((partition[1] - mu_2)/np.sqrt(sigma**2 + sigma_2**2)) - scs.norm.cdf((partition[0] - mu_2)/np.sqrt(sigma**2 + sigma_2**2)))
    
    return p1 + p2


## expert probabilities

probs_1 = np.array([[-35,-1,0,1], [-1,0,1,35], [0.2, 0.3, 0.3, 0.2]]).T ## partitions (-35, -1), (-1, 0), (0, 1), (1, 35)
probs_2 = np.array([[-35,0], [0,35], [0.45, 0.55]]).T ## partitions (-35, 0), (0, 35)
probs_3 = np.array([[-35,-1,1], [-1,1,35], [0.35, 0.3, 0.35]]).T ## partitions (-35, -1), (-1, 1), (1, 35)



probs = [probs_1, probs_2, probs_3]

prob_class = PPEProbabilities(target_type="continuous", path=False)


partitions, expert_probs = prob_class.get_expert_data(probs)


model_probs = []

for i in range(len(partitions)):
    
    probs = np.array([get_gaussian_probs(partition) for partition in partitions[i]])
    
    model_probs.append(probs)
    
    
print(expert_probs)
print(model_probs) ## quite close to the true ones with all partition sizes

[array([0.2, 0.3, 0.3, 0.2]), array([0.45, 0.55]), array([0.35, 0.3 , 0.35])]
[array([0.2893248, 0.2106752, 0.2106752, 0.2893248]), array([0.5, 0.5]), array([0.2893248, 0.4213504, 0.2893248])]


In [12]:
dir = Dirichlet(None)

print("Alpha as computed based on the given probabilities:", dir.alpha_mle(model_probs, expert_probs))  ## very high alpha as expected since this is a fixed example
print("Log likelihood when alpha is not given (computed internally):", dir.sum_llik(model_probs, expert_probs))


dir_2 = Dirichlet(10)

print("Log likelihood when alpha = 10 (fixed):", dir_2.sum_llik(model_probs, expert_probs))

Alpha as computed based on the given probabilities: 29.208767
Log likelihood when alpha is not given (computed internally): 6.697336
Log likelihood when alpha = 10 (fixed): 5.309321


### Example 2: Testing PyMC compatibility with our classes by sampling from the height growth model

In [13]:
RANDOM_SEED = 8928
rng = np.random.default_rng(RANDOM_SEED)

taus = np.array([0, 2.5, 10, 17.5]) ## the four different ages (J=4)
Y = np.array([50, 93, 141, 178])

#### Arpit

In [14]:
height_growth_model = pm.Model()

with height_growth_model:

    b = pm.Gamma("b", alpha=27.0535681555964, beta=1.46878708677914)
    
    h1 = pm.LogNormal("h1", mu=5.17689015771121, sigma=0.0108333588483753)
    htstar = pm.LogNormal("htstar", mu=5.00257693664254, sigma=0.00917685343012148)
    tstar = pm.LogNormal("tstar", mu=2.4248051249855, sigma=0.0408597528683811)
    s0 = pm.LogNormal("s0", mu=-2.62416052915936, sigma=0.0607241107175087)
    s1 = pm.LogNormal("s1", mu=0.991030985256659, sigma=1.021159164088)

    h = h1 - 2*(h1 - htstar)/(np.exp(s0*(taus - tstar)) + np.exp(s1*(taus - tstar)))
        
    Y_obs = pm.Weibull("Y_obs", alpha=b, beta=h, observed=Y)  ### In the paper the parameterization was done using mean and variance, while in PyMC it is with scale and shape. 
                                                              ### I am not sure about the correctness of the Weibull initialization here, but the sampled values are realistic and this is just a trial run.
    
    
with height_growth_model:
    idata = pm.sample_prior_predictive(random_seed=RANDOM_SEED)
    

### Expert probabilities (Height growth model, Arpit judgements)

probs_1 = np.array([[20, 40, 46, 50, 54, 58], [40, 46, 50, 54, 58, 87], [0.1, 0.15, 0.25, 0.25, 0.15, 0.1]]).T
probs_2 = np.array([[30, 60, 65, 68, 72, 75], [60, 65, 68, 72, 75, 112.5], [0.1, 0.15, 0.25, 0.25, 0.15, 0.1]]).T
probs_3 = np.array([[57.5, 115, 118, 122, 126, 128], [115, 118, 122, 126, 128, 192], [0.1, 0.15, 0.25, 0.25, 0.15, 0.1]]).T
probs_4 = np.array([[77.5, 155, 162, 170, 180, 190], [155, 162, 170, 180, 190, 285], [0.1, 0.15, 0.25, 0.25, 0.15, 0.1]]).T

probs = [probs_1, probs_2, probs_3, probs_4]

prob_class = PPEProbabilities(target_type="continuous", path=False)


partitions, expert_probs = prob_class.get_expert_data(probs)



samples_1 = idata.prior_predictive["Y_obs"][0][:,0]
samples_2 = idata.prior_predictive["Y_obs"][0][:,1]
samples_3 = idata.prior_predictive["Y_obs"][0][:,2]
samples_4 = idata.prior_predictive["Y_obs"][0][:,3]

samples = np.vstack((samples_1, samples_2, samples_3, samples_4)).T


model_probs = prob_class.ppd_probs(samples, partitions)

dir = Dirichlet(None)

print(dir.alpha_mle(model_probs, expert_probs)) ## alpha ~ 10.7, suggesting that the hyperparameter vector lambda leads to decent results! (in the R implementation it was ~9.5)

Sampling: [Y_obs, b, h1, htstar, s0, s1, tstar]


10.711594


#### Chang

In [15]:
height_growth_model = pm.Model()

with height_growth_model:

    b = pm.Gamma("b", alpha=2.70001732310141, beta=0.0649993151356405)
    
    h1 = pm.LogNormal("h1", mu=5.21564896751713, sigma=0.0178291680142866)
    htstar = pm.LogNormal("htstar", mu=5.03276720905197, sigma=0.00491627071461396)
    tstar = pm.LogNormal("tstar", mu=2.69376845516994, sigma=0.151047140645826)
    s0 = pm.LogNormal("s0", mu=-2.92277538593031, sigma=0.0986789610518291)
    s1 = pm.LogNormal("s1", mu=5.03309751276888, sigma=0.440893081072111)

    h = h1 - 2*(h1 - htstar)/(np.exp(s0*(taus - tstar)) + np.exp(s1*(taus - tstar)))
        
    Y_obs = pm.Weibull("Y_obs", alpha=b, beta=h, observed=Y)
    
with height_growth_model:
    idata = pm.sample_prior_predictive(random_seed=RANDOM_SEED)
    
    
probs_1 = np.array([[22.5, 45, 48, 51, 54, 55], [45, 48, 51, 54, 55, 82.5], [0.1, 0.15, 0.25, 0.25, 0.15, 0.1]]).T
probs_2 = np.array([[27.5, 55, 60, 65, 67, 69], [55, 60, 65, 67, 69, 103.5], [0.1, 0.15, 0.25, 0.25, 0.15, 0.1]]).T
probs_3 = np.array([[57.5, 90, 95, 100, 105, 110], [90, 95, 100, 105, 110, 165], [0.1, 0.15, 0.25, 0.25, 0.15, 0.1]]).T
probs_4 = np.array([[77.5, 160, 170, 177, 185, 190], [160, 170, 177, 185, 190, 285], [0.1, 0.15, 0.25, 0.25, 0.15, 0.1]]).T

probs = [probs_1, probs_2, probs_3, probs_4]

prob_class = PPEProbabilities(target_type="continuous", path=False)


partitions, expert_probs = prob_class.get_expert_data(probs)



samples_1 = idata.prior_predictive["Y_obs"][0][:,0]
samples_2 = idata.prior_predictive["Y_obs"][0][:,1]
samples_3 = idata.prior_predictive["Y_obs"][0][:,2]
samples_4 = idata.prior_predictive["Y_obs"][0][:,3]

samples = np.vstack((samples_1, samples_2, samples_3, samples_4)).T


model_probs = prob_class.ppd_probs(samples, partitions)

dir = Dirichlet(None)

print(dir.alpha_mle(model_probs, expert_probs))


Sampling: [Y_obs, b, h1, htstar, s0, s1, tstar]


5.116243


#### Mikko

In [16]:
height_growth_model = pm.Model()

with height_growth_model:

    b = pm.Gamma("b", alpha=14.5769504031241, beta=1.00803466606972)
    
    h1 = pm.LogNormal("h1", mu=5.16190754, sigma=0.000127077512157917)
    htstar = pm.LogNormal("htstar", mu=5.09254598, sigma=0.000939028084889941)
    tstar = pm.LogNormal("tstar", mu=2.68699688, sigma=0.06363416)
    s0 = pm.LogNormal("s0", mu=-2.1839354, sigma=0.000752964903353101)
    s1 = pm.LogNormal("s1", mu=8.2057381, sigma=2.71434539992374)

    h = h1 - 2*(h1 - htstar)/(np.exp(s0*(taus - tstar)) + np.exp(s1*(taus - tstar)))
        
    Y_obs = pm.Weibull("Y_obs", alpha=b, beta=h, observed=Y)
    
with height_growth_model:
    idata = pm.sample_prior_predictive(random_seed=RANDOM_SEED)
    
    
probs_1 = np.array([[19, 38, 45, 50, 55, 60], [38, 45, 50, 55, 60, 90], [0.1, 0.15, 0.25, 0.25, 0.15, 0.1]]).T
probs_2 = np.array([[35, 70, 80, 90, 93, 97], [70, 80, 90, 93, 97, 145.5], [0.1, 0.15, 0.25, 0.25, 0.15, 0.1]]).T
probs_3 = np.array([[57.5, 115, 120, 130, 140, 145], [115, 120, 130, 140, 145, 217.5], [0.1, 0.15, 0.25, 0.25, 0.15, 0.1]]).T
probs_4 = np.array([[77.5, 155, 165, 175, 185, 188], [155, 165, 175, 185, 188, 282], [0.1, 0.15, 0.25, 0.25, 0.15, 0.1]]).T

probs = [probs_1, probs_2, probs_3, probs_4]

prob_class = PPEProbabilities(target_type="continuous", path=False)


partitions, expert_probs = prob_class.get_expert_data(probs)



samples_1 = idata.prior_predictive["Y_obs"][0][:,0]
samples_2 = idata.prior_predictive["Y_obs"][0][:,1]
samples_3 = idata.prior_predictive["Y_obs"][0][:,2]
samples_4 = idata.prior_predictive["Y_obs"][0][:,3]

samples = np.vstack((samples_1, samples_2, samples_3, samples_4)).T


model_probs = prob_class.ppd_probs(samples, partitions)

dir = Dirichlet(None)

print(dir.alpha_mle(model_probs, expert_probs))

Sampling: [Y_obs, b, h1, htstar, s0, s1, tstar]


18.40905
